## Day 22: Reactor Reboot

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2021/tree/HEAD/notebooks/day22/puzzle.ipynb)

### Idea

Apparently, simulating the grid as a 3-dimensional array is going to take a while and a bunch of memory.

Instead, we might be able to simulate the process in terms of cuboids. But in order not to count a cube twice in case of cuboid intersection, we will split one of them into separate cuboids and not count the intersection.

### Preparation

Let's introduce some miscellaneous types and functions.

First, `Range` for a pair of coordinates.

In [ ]:
module Range = 

    type T = int * int

    let contains x (from, to') = from <= x && x <= to'

    let includes ((fromSuper, toSuper) : T) ((fromSub, toSub) : T) =
        fromSuper <= fromSub && toSub <= toSuper
        
    let overlap ((from1, to1) : T) ((from2, to2) : T) =
        (to1 < from2 || to2 < from1)
        |> not
        
    let start ((from, _) : T) = from
    let finish ((_, to') : T) = to'

    let isEmpty ((from, to') : T) =
        to' < from
    
    let len ((from, to') : T) = 
        if isEmpty (from, to') then 0
        else to' - from + 1
    

Second, three ranges define a single cuboid

In [ ]:
module Cuboid =
    type T = { 
        X: Range.T;
        Y: Range.T;
        Z: Range.T  
    }

    let fullValue c =
        int64 (Range.len c.X)
        * int64 (Range.len c.Y)
        * int64 (Range.len c.Z)
    
    let isEmpty c =
        Range.isEmpty c.X
        || Range.isEmpty c.Y
        || Range.isEmpty c.Z


module Cuboids = 
    type T = Cuboid.T list    
    let empty : T = []

Now, it is worth investing into visualizing cuboids

In [ ]:
type Mesh = { X: int list; Y: int list; Z: int list; i: int list; j: int list; k: int list}

let meshfromCuboid (cuboid : Cuboid.T) = 
    {
        X = [
            Range.start cuboid.X
            Range.finish cuboid.X + 1
            Range.start cuboid.X
            Range.finish cuboid.X + 1
            Range.start cuboid.X
            Range.finish cuboid.X + 1
            Range.start cuboid.X
            Range.finish cuboid.X + 1
        ]
        Y = [
            Range.start cuboid.Y
            Range.start cuboid.Y
            Range.finish cuboid.Y + 1
            Range.finish cuboid.Y + 1
            Range.start cuboid.Y
            Range.start cuboid.Y
            Range.finish cuboid.Y + 1
            Range.finish cuboid.Y + 1
        ]
        Z = [
            Range.start cuboid.Z
            Range.start cuboid.Z
            Range.start cuboid.Z
            Range.start cuboid.Z
            Range.finish cuboid.Z + 1
            Range.finish cuboid.Z + 1
            Range.finish cuboid.Z + 1
            Range.finish cuboid.Z + 1
        ]
        i = [0;0;0;0;0;0;3;3;4;4;7;7]
        j = [3;3;6;6;5;5;6;6;7;7;1;1]
        k = [1;2;2;4;4;1;2;7;6;5;3;5]
    }

#r "nuget: Plotly.NET, 2.0.0-preview.16"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.16"
open Plotly.NET

let private mesh3d (mesh : Mesh) = 
    Chart.Mesh3D(
        mesh.X, mesh.Y, mesh.Z,
        mesh.i, mesh.j, mesh.k,
        FlatShading = true,
        Opacity = 0.5
    )

open Microsoft.DotNet.Interactive.Formatting

Formatter.Register<Cuboid.T>(
    FormatDelegate<_>
        (fun cuboid context ->
            let chart = cuboid |> meshfromCuboid |> mesh3d
            Formatter.GetPreferredFormatterFor(typedefof<GenericChart.GenericChart>, HtmlFormatter.MimeType).Format(chart, context)
        ),
        HtmlFormatter.MimeType
    )

Formatter.Register<Cuboids.T>(
    FormatDelegate<_>
        (fun cuboids context ->
            let meshes = cuboids |> List.map meshfromCuboid
            let chart = 
                meshes
                |> List.map mesh3d
                |> Chart.combine
            Formatter.GetPreferredFormatterFor(typedefof<GenericChart.GenericChart>, HtmlFormatter.MimeType).Format(chart, context)
        ),
        HtmlFormatter.MimeType
    )


Installed Packages Plotly.NET, 2.0.0-preview.16 Plotly.NET.Interactive, 2.0.0-preview.16

### Implementation

Now we want to learn how to subtract two cuboids. Let's consider the most extreme example of mutual position of two cuboids. This is when the minuend includes the whole subtrahend into it.

In [ ]:
[
    { Cuboid.T.X = 0,9; Cuboid.T.Y = 0,9; Cuboid.T.Z = 0,9 }
    { Cuboid.T.X = 4,5; Cuboid.T.Y = 4,5; Cuboid.T.Z = 4,5 }
]

<!-- Plotly chart will be drawn inside this DIV -->

We can split one cuboid by the other one's *faces*, producing the following:

In [ ]:
[
    { Cuboid.T.X = 0,9; Cuboid.T.Y = 6,9; Cuboid.T.Z = 0,9 }
    { Cuboid.T.X = 0,9; Cuboid.T.Y = 0,3; Cuboid.T.Z = 0,9 }
    { Cuboid.T.X = 0,3; Cuboid.T.Y = 4,5; Cuboid.T.Z = 0,9 }
    { Cuboid.T.X = 6,9; Cuboid.T.Y = 4,5; Cuboid.T.Z = 0,9 }
    { Cuboid.T.X = 4,5; Cuboid.T.Y = 4,5; Cuboid.T.Z = 0,3 }
    { Cuboid.T.X = 4,5; Cuboid.T.Y = 4,5; Cuboid.T.Z = 6,9 }
]

<!-- Plotly chart will be drawn inside this DIV -->

Any of the subtrahend's faces can be either inside or outside the minuend. It seems like the coordinates of all six cuboids (at most) can be found comparing corresponding faces of two operands.

In [ ]:
module Cuboid = 
    let subtract (subtrahend : Cuboid.T) (minuend : Cuboid.T) =
        if (not <| Range.overlap subtrahend.X minuend.X
            || not <| Range.overlap subtrahend.Y minuend.Y
            || not <| Range.overlap subtrahend.Z minuend.Z) then 
            [minuend]
        else
            [
               {
                   X = minuend.X
                   Y = Range.end' subtrahend.Y + 1, Range.end' minuend.Y
                   Z = minuend.Z
               }
               {
                   X = minuend.X
                   Y = Range.start minuend.Y, Range.start subtrahend.Y - 1
                   Z = minuend.Z
               }
               {
                   X = Range.start minuend.X, Range.start subtrahend.X - 1
                   Y = max (Range.start subtrahend.Y) (Range.start minuend.Y), min (Range.end' subtrahend.Y) (Range.end' minuend.Y)
                   Z = minuend.Z
               }
               {
                   X = Range.end' subtrahend.X + 1, Range.end' minuend.X
                   Y = max (Range.start subtrahend.Y) (Range.start minuend.Y), min (Range.end' subtrahend.Y) (Range.end' minuend.Y)
                   Z = minuend.Z
               }
               {
                   X = max (Range.start subtrahend.X) (Range.start minuend.X), min (Range.end' subtrahend.X) (Range.end' minuend.X)
                   Y = max (Range.start subtrahend.Y) (Range.start minuend.Y), min (Range.end' subtrahend.Y) (Range.end' minuend.Y)
                   Z = Range.start minuend.Z, Range.start subtrahend.Z - 1
               }
               {
                   X = max (Range.start subtrahend.X) (Range.start minuend.X), min (Range.end' subtrahend.X) (Range.end' minuend.X)
                   Y = max (Range.start subtrahend.Y) (Range.start minuend.Y), min (Range.end' subtrahend.Y) (Range.end' minuend.Y)
                   Z = Range.end' subtrahend.Z + 1, Range.end' minuend.Z
               }
           ] |> List.filter (not << Cuboid.isEmpty)
    

Below are some examples of cuboids mutually positioned in different ways and their subtractions.

In [ ]:
let private subtrahend = { Cuboid.T.X = -1,2; Cuboid.T.Y = 3,6; Cuboid.T.Z = 3,6 }
let private minuend = { Cuboid.T.X = 0,5; Cuboid.T.Y = 0,5; Cuboid.T.Z = 0,5 }
[subtrahend; minuend] |> display
Cuboid.subtract subtrahend minuend

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let private subtrahend = { Cuboid.T.X = -1,7; Cuboid.T.Y = -1,0; Cuboid.T.Z = 4,6 }
let private minuend = { Cuboid.T.X = 0,5; Cuboid.T.Y = 0,5; Cuboid.T.Z = 0,5 }
[subtrahend; minuend] |> display
Cuboid.subtract subtrahend minuend

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let private subtrahend = { Cuboid.T.X = -1,7; Cuboid.T.Y = 2,3; Cuboid.T.Z = -1,1 }
let private minuend = { Cuboid.T.X = 0,5; Cuboid.T.Y = 0,5; Cuboid.T.Z = 0,5 }
[subtrahend; minuend] |> display
Cuboid.subtract subtrahend minuend

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

With subtraction implemented, addition is trivial. Also let's introduce subtraction from and addition to a list of cuboids

In [ ]:
module Cuboid = 
    let add c1 c2 =
        c1::(Cuboid.subtract c1 c2)


module Cuboids = 
    let add c cs : Cuboids.T =
        c :: (List.collect (fun x -> Cuboid.subtract c x)) cs
            
    let subtract c cs : Cuboids.T =
        (List.collect (fun x -> Cuboid.subtract c x)) cs

The small example from the problem statement:

In [ ]:
let r1 = { Cuboid.T.X = 10,12; Cuboid.T.Y = 10,12; Cuboid.T.Z = 10,12}
let r2 = { Cuboid.T.X = 11,13; Cuboid.T.Y = 11,13; Cuboid.T.Z = 11,13}
let r3 = { Cuboid.T.X = 9,11; Cuboid.T.Y = 9,11; Cuboid.T.Z = 9,11}
let r4 = { Cuboid.T.X = 10,10; Cuboid.T.Y = 10,10; Cuboid.T.Z = 10,10}

Cuboids.empty
|> Cuboids.add r1
|> Cuboids.add r2
|> Cuboids.subtract r3
|> Cuboids.add r4

<!-- Plotly chart will be drawn inside this DIV -->

### Part 1

In [ ]:
#r "nuget:FSharp.Text.RegexProvider"
open FSharp.Text.RegexProvider
open FSharp.Text.RegexExtensions
type private TypeInputRegex =
   Regex< @"(?<Op>\w+)\sx=(?<FromX>-?\d+)\.\.(?<ToX>-?\d+),y=(?<FromY>-?\d+)\.\.(?<ToY>-?\d+),z=(?<FromZ>-?\d+)\.\.(?<ToZ>-?\d+)" >

let parseLine line =
    match TypeInputRegex().TryTypedMatch(line) with
    | None -> failwith "unparsed"
    | Some match' ->
        (match'.Op.Value,
                        { Cuboid.T.X = match'.FromX.AsInt, match'.ToX.AsInt
                          Cuboid.T.Y = match'.FromY.AsInt, match'.ToY.AsInt
                          Cuboid.T.Z = match'.FromZ.AsInt, match'.ToZ.AsInt })

Installed Packages FSharp.Text.RegexProvider, 2.1.0

In [ ]:
#!value --name sample1Raw --from-file=./sample1

In [ ]:
#!value --name sample2Raw --from-file=./sample2

In [ ]:
#!value --name inputRaw --from-file=./data

In [ ]:
#!share sample1Raw --from value
#!share sample2Raw --from value
#!share inputRaw --from value

#load "../common.fsx"
let sample1Input = sample1Raw |> readLines |> Array.map parseLine
let sample2Input = sample2Raw |> readLines |> Array.map parseLine
let actualInput = inputRaw |> readLines |> Array.map parseLine

In [ ]:
let perform commands = 
    commands
    |> Array.fold (fun cuboids (op, cuboid) -> 
                        (cuboid, cuboids) 
                        ||> match op with 
                            | "on" -> Cuboids.add 
                            | "off" -> Cuboids.subtract) Cuboids.empty
     
 

In [ ]:
sample1Input
|> Array.filter (fun (_, c) -> 
                    Range.includes (-50,50) c.X
                    && Range.includes (-50,50) c.Y
                    && Range.includes (-50,50) c.Z
                    )
|> perform
|> displayPipe
|> List.sumBy Cuboid.fullValue

<!-- Plotly chart will be drawn inside this DIV -->

590784

In [ ]:
actualInput
|> Array.filter (fun (_, c) -> 
                    Range.includes (-50,50) c.X
                    && Range.includes (-50,50) c.Y
                    && Range.includes (-50,50) c.Z
                    )
|> perform
|> List.sumBy Cuboid.fullValue

580012

### Part 2

In [ ]:
sample2Input
|> perform
|> List.sumBy Cuboid.fullValue

2758514936282235

In [ ]:
actualInput
|> perform
|> List.sumBy Cuboid.fullValue

1334238660555542